In [1]:
import numpy as np
import pandas as pd
from pathlib import Path

In [4]:
# PART 1 — Setup & Load the dataset

DATA_PATH = Path("flight_1yr.csv")

assert DATA_PATH.exists(), f"Dataset not found at: {DATA_PATH.resolve()}"

df = pd.read_csv(DATA_PATH)

print("Shape:", df.shape)
print("\nColumns:", list(df.columns))

Shape: (736124, 17)

Columns: ['FL_DATE', 'AIRLINE', 'FL_NUMBER', 'ORIGIN_CITY', 'DEST_CITY', 'DEP_DELAY', 'ARR_DELAY', 'AIR_TIME', 'DISTANCE', 'DELAY_DUE_CARRIER', 'DELAY_DUE_WEATHER', 'DELAY_DUE_NAS', 'DELAY_DUE_SECURITY', 'DELAY_DUE_LATE_AIRCRAFT', 'WeekdayName', 'Month', 'DELAY_REASONS']


In [3]:
display(df.head())

,FL_DATE,AIRLINE,FL_NUMBER,ORIGIN_CITY,DEST_CITY,DEP_DELAY,ARR_DELAY,AIR_TIME,DISTANCE,DELAY_DUE_CARRIER,DELAY_DUE_WEATHER,DELAY_DUE_NAS,DELAY_DUE_SECURITY,DELAY_DUE_LATE_AIRCRAFT,WeekdayName,Month,DELAY_REASONS
0,11/19/22,Delta Air Lines Inc.,1149,"Minneapolis, MN","Seattle, WA",-6,-5.0,189.0,1399,0,0,0,0,0,Saturday,2022-11,NOT DELAY
1,3/6/23,Delta Air Lines Inc.,2295,"Minneapolis, MN","San Francisco, CA",-1,24.0,249.0,1589,0,0,24,0,0,Monday,2023-03,NAS
2,6/11/23,American Airlines Inc.,2134,"Washington, DC","Boston, MA",-9,-29.0,58.0,399,0,0,0,0,0,Sunday,2023-06,NOT DELAY
3,2/12/23,Spirit Air Lines,590,"Houston, TX","Los Angeles, CA",-3,-11.0,200.0,1379,0,0,0,0,0,Sunday,2023-02,NOT DELAY
4,11/12/22,Delta Air Lines Inc.,2706,"Grand Rapids, MI","Minneapolis, MN",-10,-30.0,61.0,408,0,0,0,0,0,Saturday,2022-11,NOT DELAY


In [6]:
# PART 2 — Create target columns 
df = df.copy()

In [7]:
# 1) Target label 
assert "ARR_DELAY" in df.columns, "ARR_DELAY column missing."
df["DELAY_FLAG"] = (pd.to_numeric(df["ARR_DELAY"], errors="coerce") >= 15).astype("int8")


In [8]:
# 2) Route string for Tableau 
assert {"ORIGIN_CITY", "DEST_CITY"}.issubset(df.columns), "Need ORIGIN_CITY and DEST_CITY."
df["ROUTE"] = df["ORIGIN_CITY"].astype(str) + " \u2192 " + df["DEST_CITY"].astype(str)  # → arrow


In [9]:
# 3) Parse dates & derive parts
assert "FL_DATE" in df.columns, "FL_DATE column missing."
df["FL_DATE"] = pd.to_datetime(df["FL_DATE"], errors="coerce")
df["YEAR"] = df["FL_DATE"].dt.year.astype("Int16")
df["MONTH_NUM"] = df["FL_DATE"].dt.month.astype("Int8")
df["DAY_OF_MONTH"] = df["FL_DATE"].dt.day.astype("Int8")


In [10]:
# 4) Ensure DISTANCE is numeric
if "DISTANCE" in df.columns:
    df["DISTANCE"] = pd.to_numeric(df["DISTANCE"], errors="coerce")


In [11]:
# 5) Quick checks
print("Overall delay rate (ARR_DELAY>=15):", round(df["DELAY_FLAG"].mean(), 4))
print("Rows with unparseable FL_DATE:", int(df["FL_DATE"].isna().sum()))
print("Sample columns preview:")
display(df[["FL_DATE","YEAR","MONTH_NUM","DAY_OF_MONTH","WeekdayName",
            "AIRLINE","ORIGIN_CITY","DEST_CITY","ROUTE","DISTANCE",
            "ARR_DELAY","DELAY_FLAG"]].head(10))

Overall delay rate (ARR_DELAY>=15): 0.2187
Rows with unparseable FL_DATE: 0
Sample columns preview:


,FL_DATE,YEAR,MONTH_NUM,DAY_OF_MONTH,WeekdayName,AIRLINE,ORIGIN_CITY,DEST_CITY,ROUTE,DISTANCE,ARR_DELAY,DELAY_FLAG
0,2022-11-19,2022,11,19,Saturday,Delta Air Lines Inc.,"Minneapolis, MN","Seattle, WA","Minneapolis, MN → Seattle, WA",1399,-5.0,0
1,2023-03-06,2023,3,6,Monday,Delta Air Lines Inc.,"Minneapolis, MN","San Francisco, CA","Minneapolis, MN → San Francisco, CA",1589,24.0,1
2,2023-06-11,2023,6,11,Sunday,American Airlines Inc.,"Washington, DC","Boston, MA","Washington, DC → Boston, MA",399,-29.0,0
3,2023-02-12,2023,2,12,Sunday,Spirit Air Lines,"Houston, TX","Los Angeles, CA","Houston, TX → Los Angeles, CA",1379,-11.0,0
4,2022-11-12,2022,11,12,Saturday,Delta Air Lines Inc.,"Grand Rapids, MI","Minneapolis, MN","Grand Rapids, MI → Minneapolis, MN",408,-30.0,0
5,2022-09-06,2022,9,6,Tuesday,American Airlines Inc.,"Dallas/Fort Worth, TX","Pensacola, FL","Dallas/Fort Worth, TX → Pensacola, FL",604,6.0,0
6,2023-04-15,2023,4,15,Saturday,Republic Airline,"Minneapolis, MN","Newark, NJ","Minneapolis, MN → Newark, NJ",1008,1124.0,1
7,2023-01-03,2023,1,3,Tuesday,Allegiant Air,"Concord, NC","Sanford, FL","Concord, NC → Sanford, FL",457,2.0,0
8,2023-07-23,2023,7,23,Sunday,United Air Lines Inc.,"Austin, TX","Houston, TX","Austin, TX → Houston, TX",140,-2.0,0
9,2022-10-08,2022,10,8,Saturday,American Airlines Inc.,"Dallas/Fort Worth, TX","Tucson, AZ","Dallas/Fort Worth, TX → Tucson, AZ",813,-13.0,0


In [16]:
df_model = df.copy()

In [24]:
from pathlib import Path
import json

In [17]:
DATA_DIR = Path("data")
DATA_DIR.mkdir(parents=True, exist_ok=True)
PART2_CSV = DATA_DIR / "flight_part2_ready.csv" 

In [18]:
#Define the columns to keep for modeling/dashboard 
LABEL_COL       = "DELAY_FLAG"
LOW_CARD_COLS   = ["AIRLINE", "WeekdayName"]
HIGH_CARD_COLS  = ["ORIGIN_CITY", "DEST_CITY", "ROUTE"]
NUM_COLS        = ["DISTANCE", "MONTH_NUM"]
DATE_PARTS      = ["FL_DATE", "YEAR", "MONTH_NUM", "DAY_OF_MONTH"]
CARRY_ALONG     = []  

In [19]:
# keep FL_NUMBER / Month if they exist
if "FL_NUMBER" in df_model.columns: CARRY_ALONG.append("FL_NUMBER")
if "Month" in df_model.columns:     CARRY_ALONG.append("Month")

KEEP_COLS = list(dict.fromkeys(
    DATE_PARTS + LOW_CARD_COLS + HIGH_CARD_COLS + NUM_COLS + [LABEL_COL] + CARRY_ALONG
))


In [20]:
# Build the saved frame (df_part2)
df_part2 = df_model[KEEP_COLS].copy()
df_part2["FL_DATE"]     = pd.to_datetime(df_part2["FL_DATE"], errors="coerce")
df_part2["YEAR"]        = df_part2["YEAR"].astype("Int16")
df_part2["MONTH_NUM"]   = df_part2["MONTH_NUM"].astype("Int8")
df_part2["DAY_OF_MONTH"]= df_part2["DAY_OF_MONTH"].astype("Int8")
df_part2["DISTANCE"]    = pd.to_numeric(df_part2["DISTANCE"], errors="coerce")
df_part2["DELAY_FLAG"]  = df_part2["DELAY_FLAG"].astype("int8")


In [22]:

# Save CSV (always)
df_part2.to_csv(PART2_CSV, index=False)
try:
    df_part2.to_parquet(PART2_PQ, index=False)  
    saved_parquet = True
except Exception:
    saved_parquet = False

In [25]:
#Save a tiny metadata file so later parts can reuse the same config
META = {
    "label_col": LABEL_COL,
    "low_card_cols": LOW_CARD_COLS,
    "high_card_cols": HIGH_CARD_COLS,
    "num_cols": NUM_COLS,
    "date_parts": DATE_PARTS,
    "carry_along": CARRY_ALONG,
    "files": {
        "csv": str(PART2_CSV),
        "parquet": str(PART2_PQ) if saved_parquet else None
    },
    "rows": int(len(df_part2)),
    "delay_rate": float(df_part2[LABEL_COL].mean())
}
with open(DATA_DIR / "flight_part2_meta.json", "w") as f:
    json.dump(META, f, indent=2)


In [27]:
print("Rows:", len(df_part2), " | Delay rate:", round(META["delay_rate"], 4))
print("CSV:", PART2_CSV)


Rows: 736124  | Delay rate: 0.2187
CSV: data/flight_part2_ready.csv
